In [122]:
from heapq import heapify, heappush, heappop


def read_board(file="input.txt"):
    board = []
    with open(file) as f:
        for line in f:
            board.append(list(line.strip()))
            for i in range(len(board[-1])):
                if board[-1][i] == "E":
                    end = (len(board) - 1, i)
                elif board[-1][i] == "S":
                    start = (len(board) - 1, i)
    return board, start, end


def backtrack_path(y, x, prev, start):
    paths = []
    stack = [([(y, x)], (y, x))]  # Stack of (current_path, current_node)
    while stack:
        current_path, (cy, cx) = stack.pop()
        if (cy, cx) == start:
            paths.append(current_path)
        else:
            for py, px in prev[cy][cx]:
                stack.append(([(py, px)] + current_path, (py, px)))
    return paths


def dijkstra_solve(file="input.txt"):
    board, start, end = read_board(file)
    steps = [[float("inf")] * len(board[0]) for _ in range(len(board))]
    prev = [[set() for _ in range(len(board[0]))] for _ in range(len(board))]
    steps[start[0]][start[1]] = 0
    min_steps = float("inf")
    heapify(heap := [(0, start[1], start[0])])
    while heap:
        curr_step, x, y = heappop(heap)
        if curr_step > min_steps:
            break
        if (y, x) == end:
            min_steps = curr_step
            continue
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if board[ny][nx] == "." or board[ny][nx] == "E":
                if steps[ny][nx] > curr_step + 1:
                    steps[ny][nx] = curr_step + 1
                    prev[ny][nx].add((y, x))
                    heappush(heap, (curr_step + 1, nx, ny))
    best_paths = backtrack_path(*end, prev, start)
    unique_paths = list({tuple(path) for path in best_paths})
    path_cells = set()
    for path in unique_paths:
        for i, (ry, rx) in enumerate(path):
            path_cells.add((ry, rx, i))
        path_cells.add((start[0], start[1], 0))

    for ry, rx, i in path_cells:
        board[ry][rx] = i

    return board, best_paths

In [123]:
# Q1
board, best_paths = dijkstra_solve()
cheat_set = set()
steps_save = 0
for y, x in best_paths[0]:
    for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        nx, ny = x + 2 * dx, y + 2 * dy
        if 0 <= nx < len(board[0]) and 0 <= ny < len(board) and board[ny][nx] != "#":
            steps_saved = board[ny][nx] - board[y][x] - 2
            if steps_saved >= 100 and ((y, x), (ny, nx)) not in cheat_set:
                cheat_set.add(((y, x), (ny, nx), steps_saved))
                steps_save += 1
print(steps_save)

1263


In [ ]:
# Q2
from tqdm import tqdm
board, best_paths = dijkstra_solve("input.txt")
cheats = 0
best_path = list(best_paths[0])
for start in tqdm(range(len(best_path))):
    for end in range(start + 1, len(best_path)):
        (y1, x1), (y2, x2) = best_path[start], best_path[end]
        dist = abs(y1 - y2) + abs(x1 - x2)
        cost = board[y2][x2] - board[y1][x1]
        if dist <= 20 and cost - dist >= 100:
            cheats += 1
print(cheats)

100%|██████████| 9325/9325 [00:32<00:00, 283.96it/s] 

957831
